In [1]:
import os
import time
import pandas as pd 
import requests
import urllib3
import json
import sys
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime, timedelta

pd.options.display.max_colwidth = 1000
pd.set_option('display.max_columns', None)

### Files Yara Rules

In [2]:
files_yaraRules_dir = 'C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\PostDataEnrichment\\Step2\\YaraRules.csv'

In [3]:
df_yaraRules = pd.read_csv(files_yaraRules_dir, low_memory = False)
df_yaraRules = df_yaraRules.drop(columns=['Unnamed: 0'])
df_yaraRules

,sha1,sha256,md5,yara_rule_name,yara_rule_description,yara_rule_author,area
0,01facbf9132aea6e12300d19119f434a7079f8f0,7c911b4d457b1b6124a1211cd4737c056e9a12dc930a05edfe7289cbed11e86c,f41a14859f324f4b208163475c347837,asyncrat,detect asyncrat in memory,JPCERT/CC Incident Response Group,Network Activity
1,01facbf9132aea6e12300d19119f434a7079f8f0,7c911b4d457b1b6124a1211cd4737c056e9a12dc930a05edfe7289cbed11e86c,f41a14859f324f4b208163475c347837,win asyncrat j1,detects asyncrat,Johannes Bader @viql,Network Activity
2,0306b6dcf5035b9e6759e4573f629e4ba7191395,8499e016c470f2735d4873be23619a68ce08a7a1f67b7253f9da698d3d688655,6db93aef69435a63810b7cffec6b30d2,chrome stealer bin mem,chrome in files like avemaria,James_inthe_box,Data Activity
3,0306b6dcf5035b9e6759e4573f629e4ba7191395,8499e016c470f2735d4873be23619a68ce08a7a1f67b7253f9da698d3d688655,6db93aef69435a63810b7cffec6b30d2,indicator suspicious exe uacbypass eventviewer,detects windows exceutables potentially bypassing uac using eventvwr.exe,ditekSHen,Network Activity
4,0306b6dcf5035b9e6759e4573f629e4ba7191395,8499e016c470f2735d4873be23619a68ce08a7a1f67b7253f9da698d3d688655,6db93aef69435a63810b7cffec6b30d2,keylog bin mem,contains keylog,James_inthe_box,Keylogging Activity
...,...,...,...,...,...,...,...
2382,ff91376da89e90d19e7c9f53dd8915be22e83627,d9dcaa71fb3a6eb36123628d089cfa59f1b3f01120626a5e4d791a7adf4116ab,31b76b0b5ab72c603ef54efa63c32b22,indicator suspicious stomped pecompilation timestamp inthefuture,detect executables with stomped pe compilation timestamp that is greater than local current time,ditekSHen,Network Activity
2383,ff91376da89e90d19e7c9f53dd8915be22e83627,d9dcaa71fb3a6eb36123628d089cfa59f1b3f01120626a5e4d791a7adf4116ab,31b76b0b5ab72c603ef54efa63c32b22,nanocore,detect nanocore in memory,JPCERT/CC Incident Response Group,Network Activity
2384,ff91376da89e90d19e7c9f53dd8915be22e83627,d9dcaa71fb3a6eb36123628d089cfa59f1b3f01120626a5e4d791a7adf4116ab,31b76b0b5ab72c603ef54efa63c32b22,nanocore rat feb18 1,detects nanocore rat,Florian Roth,Network Activity
2385,ff91376da89e90d19e7c9f53dd8915be22e83627,d9dcaa71fb3a6eb36123628d089cfa59f1b3f01120626a5e4d791a7adf4116ab,31b76b0b5ab72c603ef54efa63c32b22,nanocore rat gen 2,detetcs the nanocore rat,Florian Roth,unknown


In [4]:
df_yaraRules['yara_rule_name'] = df_yaraRules['yara_rule_name'].fillna(' ')
df_yaraRules['yara_rule_description'] = df_yaraRules['yara_rule_description'].fillna(' ')
df_yaraRules

,sha1,sha256,md5,yara_rule_name,yara_rule_description,yara_rule_author,area
0,01facbf9132aea6e12300d19119f434a7079f8f0,7c911b4d457b1b6124a1211cd4737c056e9a12dc930a05edfe7289cbed11e86c,f41a14859f324f4b208163475c347837,asyncrat,detect asyncrat in memory,JPCERT/CC Incident Response Group,Network Activity
1,01facbf9132aea6e12300d19119f434a7079f8f0,7c911b4d457b1b6124a1211cd4737c056e9a12dc930a05edfe7289cbed11e86c,f41a14859f324f4b208163475c347837,win asyncrat j1,detects asyncrat,Johannes Bader @viql,Network Activity
2,0306b6dcf5035b9e6759e4573f629e4ba7191395,8499e016c470f2735d4873be23619a68ce08a7a1f67b7253f9da698d3d688655,6db93aef69435a63810b7cffec6b30d2,chrome stealer bin mem,chrome in files like avemaria,James_inthe_box,Data Activity
3,0306b6dcf5035b9e6759e4573f629e4ba7191395,8499e016c470f2735d4873be23619a68ce08a7a1f67b7253f9da698d3d688655,6db93aef69435a63810b7cffec6b30d2,indicator suspicious exe uacbypass eventviewer,detects windows exceutables potentially bypassing uac using eventvwr.exe,ditekSHen,Network Activity
4,0306b6dcf5035b9e6759e4573f629e4ba7191395,8499e016c470f2735d4873be23619a68ce08a7a1f67b7253f9da698d3d688655,6db93aef69435a63810b7cffec6b30d2,keylog bin mem,contains keylog,James_inthe_box,Keylogging Activity
...,...,...,...,...,...,...,...
2382,ff91376da89e90d19e7c9f53dd8915be22e83627,d9dcaa71fb3a6eb36123628d089cfa59f1b3f01120626a5e4d791a7adf4116ab,31b76b0b5ab72c603ef54efa63c32b22,indicator suspicious stomped pecompilation timestamp inthefuture,detect executables with stomped pe compilation timestamp that is greater than local current time,ditekSHen,Network Activity
2383,ff91376da89e90d19e7c9f53dd8915be22e83627,d9dcaa71fb3a6eb36123628d089cfa59f1b3f01120626a5e4d791a7adf4116ab,31b76b0b5ab72c603ef54efa63c32b22,nanocore,detect nanocore in memory,JPCERT/CC Incident Response Group,Network Activity
2384,ff91376da89e90d19e7c9f53dd8915be22e83627,d9dcaa71fb3a6eb36123628d089cfa59f1b3f01120626a5e4d791a7adf4116ab,31b76b0b5ab72c603ef54efa63c32b22,nanocore rat feb18 1,detects nanocore rat,Florian Roth,Network Activity
2385,ff91376da89e90d19e7c9f53dd8915be22e83627,d9dcaa71fb3a6eb36123628d089cfa59f1b3f01120626a5e4d791a7adf4116ab,31b76b0b5ab72c603ef54efa63c32b22,nanocore rat gen 2,detetcs the nanocore rat,Florian Roth,unknown


### NLP sul Behaviours in ogni file

#### Example of NLP

In [5]:
!python -m spacy download it_core_news_sm


     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/13.0 MB ? eta -:--:--
     --------------------------------------- 0.0/13.0 MB 330.3 kB/s eta 0:00:40
     --------------------------------------- 0.1/13.0 MB 573.4 kB/s eta 0:00:23
      --------------------------------------- 0.2/13.0 MB 1.1 MB/s eta 0:00:12
      --------------------------------------- 0.3/13.0 MB 1.3 MB/s eta 0:00:10
     - -------------------------------------- 0.3/13.0 MB 1.4 MB/s eta 0:00:10
     - -------------------------------------- 0.4/13.0 MB 1.2 MB/s eta 0:00:11
     - -------------------------------------- 0.5/13.0 MB 1.3 MB/s eta 0:00:10
     - -------------------------------------- 0.5/13.0 MB 1.3 MB/s eta 0:00:10
     - -------------------------------------- 0.6/13.0 MB 1.4 MB/s eta 0:00:09
     -- ------------------------------------- 0.7/13.0 MB 1.5 MB/s eta 

In [6]:
import spacy

nlp = spacy.load("it_core_news_sm")

frase = "Il gatto mangia il pesce."

doc = nlp(frase)

# Estrai sostantivi e verbi dalla frase
parole_di_interesse = [token.text for token in doc if token.pos_ in ['NOUN', 'VERB']]

# Stampa i risultati
print("Parole di interesse:", parole_di_interesse)

Parole di interesse: ['gatto', 'mangia', 'pesce']


#### Get Words from descriptions

In [7]:
pip install spacy


Note: you may need to restart the kernel to use updated packages.


In [8]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 220.2 kB/s eta 0:00:59
     --------------------------------------- 0.1/12.8 MB 365.7 kB/s eta 0:00:35
     --------------------------------------- 0.1/12.8 MB 602.4 kB/s eta 0:00:22
     --------------------------------------- 0.1/12.8 MB 602.4 kB/s eta 0:00:22
     --------------------------------------- 0.1/12.8 MB 602.4 kB/s eta 0:00:22
     --------------------------------------- 0.1/12.8 MB 602.4 kB/s eta 0:00:22
      -------------------------------------- 0.2/12.8 MB 541.9 kB/s eta 0:00:24
      -------------------------------------- 0.3/12.8 MB 731.4 kB/s eta 0:00:18
     - ------------------------------------- 0.5/12.8 MB 972.0 kB/s eta 0:00:13
     - -------------------------------------- 0.6/12.8 MB 1.2 MB

In [31]:
from tqdm import tqdm

def create_words_df(df):
    nlp = spacy.load("en_core_web_sm")
    combined_texts = df['yara_rule_name'] + '. ' + df['yara_rule_description']
    
    data_list = []
    
    for i, text in tqdm(enumerate(combined_texts), total=len(combined_texts), desc="Processing documents"):
        doc = nlp(text)
        words_interesting = [token.text for token in doc if token.pos_ in ['NOUN', 'VERB']]
        
        for word in words_interesting:
            word_type = 'noun' if nlp(word)[0].pos_ == 'NOUN' else 'verb'
            data_list.append({'word': word, 'type': word_type, 'index': i})
            
    dataframe = pd.DataFrame(data_list).reset_index()
    
    return dataframe


In [32]:
words_df = create_words_df(df_yaraRules)

Processing documents: 100%|██████████| 2387/2387 [01:16<00:00, 31.15it/s]


In [33]:
words_df

,level_0,word,type,index
0,0,asyncrat,noun,0
1,1,detect,verb,0
2,2,asyncrat,noun,0
3,3,memory,noun,0
4,4,detects,noun,1
...,...,...,...,...
9838,9838,memory,noun,2383
9839,9839,detects,noun,2384
9840,9840,detetcs,verb,2385
9841,9841,rat,verb,2385


In [34]:
words_df.to_csv(r'C:\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\ER_Schema\\Words_YaraRules.csv')

In [35]:
type(words_df)

pandas.core.frame.DataFrame

In [36]:
merged_words_df = pd.merge(df_yaraRules, words_df, left_index=True, right_on='index')
merged_words_df = merged_words_df.drop(columns=['index', 'level_0'])
merged_words_df


,sha1,sha256,md5,yara_rule_name,yara_rule_description,yara_rule_author,area,word,type
0,01facbf9132aea6e12300d19119f434a7079f8f0,7c911b4d457b1b6124a1211cd4737c056e9a12dc930a05edfe7289cbed11e86c,f41a14859f324f4b208163475c347837,asyncrat,detect asyncrat in memory,JPCERT/CC Incident Response Group,Network Activity,asyncrat,noun
1,01facbf9132aea6e12300d19119f434a7079f8f0,7c911b4d457b1b6124a1211cd4737c056e9a12dc930a05edfe7289cbed11e86c,f41a14859f324f4b208163475c347837,asyncrat,detect asyncrat in memory,JPCERT/CC Incident Response Group,Network Activity,detect,verb
2,01facbf9132aea6e12300d19119f434a7079f8f0,7c911b4d457b1b6124a1211cd4737c056e9a12dc930a05edfe7289cbed11e86c,f41a14859f324f4b208163475c347837,asyncrat,detect asyncrat in memory,JPCERT/CC Incident Response Group,Network Activity,asyncrat,noun
3,01facbf9132aea6e12300d19119f434a7079f8f0,7c911b4d457b1b6124a1211cd4737c056e9a12dc930a05edfe7289cbed11e86c,f41a14859f324f4b208163475c347837,asyncrat,detect asyncrat in memory,JPCERT/CC Incident Response Group,Network Activity,memory,noun
4,01facbf9132aea6e12300d19119f434a7079f8f0,7c911b4d457b1b6124a1211cd4737c056e9a12dc930a05edfe7289cbed11e86c,f41a14859f324f4b208163475c347837,win asyncrat j1,detects asyncrat,Johannes Bader @viql,Network Activity,detects,noun
...,...,...,...,...,...,...,...,...,...
9838,ff91376da89e90d19e7c9f53dd8915be22e83627,d9dcaa71fb3a6eb36123628d089cfa59f1b3f01120626a5e4d791a7adf4116ab,31b76b0b5ab72c603ef54efa63c32b22,nanocore,detect nanocore in memory,JPCERT/CC Incident Response Group,Network Activity,memory,noun
9839,ff91376da89e90d19e7c9f53dd8915be22e83627,d9dcaa71fb3a6eb36123628d089cfa59f1b3f01120626a5e4d791a7adf4116ab,31b76b0b5ab72c603ef54efa63c32b22,nanocore rat feb18 1,detects nanocore rat,Florian Roth,Network Activity,detects,noun
9840,ff91376da89e90d19e7c9f53dd8915be22e83627,d9dcaa71fb3a6eb36123628d089cfa59f1b3f01120626a5e4d791a7adf4116ab,31b76b0b5ab72c603ef54efa63c32b22,nanocore rat gen 2,detetcs the nanocore rat,Florian Roth,unknown,detetcs,verb
9841,ff91376da89e90d19e7c9f53dd8915be22e83627,d9dcaa71fb3a6eb36123628d089cfa59f1b3f01120626a5e4d791a7adf4116ab,31b76b0b5ab72c603ef54efa63c32b22,nanocore rat gen 2,detetcs the nanocore rat,Florian Roth,unknown,rat,verb


In [37]:
merged_words_df.to_csv(r'C:\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\FileAnalysis\\ER_Schema\\Words_YaraRules.csv')